In [ ]:
from pdrtpy.measurement import Measurement
from pdrtpy.modelset import ModelSet
from pdrtpy.plot.modelplot import ModelPlot
import pdrtpy.pdrutils as utils
from pdrtpy.tool.lineratiofit import LineRatioFit
from pdrtpy.plot.lineratioplot import LineRatioPlot
from astropy.nddata import StdDevUncertainty
import astropy.units as u
import numpy as np
#from lmfit import Model, Parameters, Minimizer, minimize, fit_report
import corner

In [ ]:
from pdrtpy import version
version()

### Data for pillar

In [ ]:
myunit = "erg s-1 cm-2 sr-1" 
intensity = {
    "CII_158": 1.2E-3,
    "CO_32": 4.3E-6,
    "OI_63": 6.4E-3,
    "FIR": 0.9
}
error = {
    "CII_158": 1.83E-5,
    "CO_32": 2.54E-8,
    "OI_63": 3.30E-4,
    "FIR": 0.2*intensity["FIR"]  #guess 20%??
}
a = []
for k in intensity:
    a.append(Measurement(data=intensity[k],uncertainty = StdDevUncertainty(error[k]),identifier=k,unit=myunit))
for m in a:
    print(f'{m.id:>7s}  {m:3.2e}')

## Mark has new WK 2020 models as of 10/15

In [ ]:
ms = ModelSet("wk2020",z=1)
ms.table.show_in_notebook()

### Do the fit with standard least squares, which is the default *method* to run()

In [ ]:
p = LineRatioFit(ms,a)
p.run()
print(f' n = {p.density:.2e}\nG0 = {p.radiation_field:.2e}')

### The fit resuls are stored in a *FitMap*, where each pixel contains a instance of an *MinimizerResult*.   
See https://lmfit-py.readthedocs.io/en/latest/fitting.html#minimizerresult-the-optimization-result

Since this is a single pixel example, the result is in index 0.

In [ ]:
p.fit_result[0]

In [ ]:
plot = LineRatioPlot(p)
plot.reduced_chisq(legend=True,norm='log')

In [ ]:
plot.overlay_all_ratios(measurements=None)

In [ ]:
plot.ratios_on_models(image=True,ncols=2,meas_color=['#4daf4a'],norm='log')

### Let's get all Bayesian up in this crib
Instead of LSQ, you can use the emcee package to do Monte Carlo Markov Chain analysis to determine n and G0.
Note: this takes a few minutes.

*Caution: Do not use this method for maps!*

In [ ]:
p.run(method="emcee")

In [ ]:
res = p.fit_result[0]
fig = corner.corner(res.flatchain, labels=res.var_names, truths=list(res.params.valuesdict().values()))

### Data for p1

In [ ]:
myunit = "erg s-1 cm-2 sr-1" 
intensity = {
    "CII_158": 1.2E-3,
    "CO_32": 4.3E-6,
    "OI_63": 6.4E-3,
    "FIR": 0.9,
    "H200S1":6.6E-5,
    "H200S2":1.5E-4
}
error = {
    "CII_158": 1.83E-5,
    "CO_32": 2.54E-8,
    "OI_63": 3.30E-4,
    "FIR": 0.2*intensity["FIR"],  #guess 20%??
    "H200S1": 8.7E-6,
    "H200S2": 1.26E-5
}
a = []
for k in intensity:
    a.append(Measurement(data=intensity[k],uncertainty = StdDevUncertainty(error[k]),identifier=k,unit=myunit))
for m in a:
    print(f'{m.id:>7s}  {m:3.2e}')

In [ ]:
p1 = LineRatioFit(ms,a)
p1.run()
print(f' n = {p1.density:.2e}\nG0 = {p1.radiation_field:.2e}')
p1.fit_result[0]

In [ ]:
plot1 = LineRatioPlot(p1)
plot1.reduced_chisq(legend=True,norm='log')

In [ ]:
plot1.overlay_all_ratios(measurements=None)

In [ ]:
plot1.ratios_on_models(image=True,ncols=2,meas_color=['#4daf4a'],norm='log')

### You can pass some emcee keywords to *run()*
More steps means a longer time to compute, but will ensure the chain is long enough.

See https://lmfit-py.readthedocs.io/en/latest/fitting.html#lmfit.minimizer.Minimizer.emcee

In [ ]:
p1.run(method="emcee", steps=2050, burn=50)

#### You can modify the corner plot, changing the labels or using other available keywords.
See https://corner.readthedocs.io/en/latest/api.html

In [ ]:
res = p1.fit_result[0]
fig = corner.corner(res.flatchain, labels=['n','$G_0$'], truths=list(res.params.valuesdict().values()))